In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd
import dask

from tqdm import tqdm
import jax
jax.config.update('jax_platform_name', 'cpu')
# jax.config.update('jax_log_compiles', True)
# jax.config.update("jax_debug_nans", True)
# jax.config.update("jax_enable_x64", True)

In [ ]:


sys.path.append("../..")

from lib import utils as U
from lib.ehr.dataset import load_dataset, load_dataset_scheme, load_dataset_config, Dataset
from lib.ehr.interface import Patients, InterfaceConfig
from lib.ehr.concepts import DemographicVectorConfig


In [ ]:
import logging
logging.root.level = logging.INFO

In [ ]:
tag = 'M3'
PATH = f'{os.environ.get("HOME")}/GP/ehr-data/mimic3-cohort'
sample = None
cache =  f'cached_inteface/patients_{tag}'
dataset_config = load_dataset_config(tag, 
                                     sample=sample,
                                     path=PATH)
dataset_gen = lambda c: load_dataset(config=c)

##### Possible Interface Scheme Configurations

In [ ]:
import json
dataset_scheme = load_dataset_scheme(tag)
interface_schem_options = dataset_scheme.supported_target_scheme_options
print(json.dumps(interface_schem_options, sort_keys=True, indent=4))

In [ ]:
interface_scheme = dataset_scheme.make_target_scheme_config(dx='DxICD9',
                                                            outcome='dx_icd9_filter_v3_groups',
                                                            ethnicity='MIMIC3Eth7')


# Demographic vector attributes
demographic_vector_conf = DemographicVectorConfig(
    age=False, 
    gender=False, 
    ethnicity=False
)
interface_config = InterfaceConfig(scheme=interface_scheme,
                                   dataset_scheme=dataset_scheme,
                                   demographic_vector=demographic_vector_conf,
                                   cache=cache)

In [ ]:
interface_config

In [ ]:
m3patients = Patients.try_load_cached(interface_config,
                                      dataset_config=dataset_config,
                                      dataset_generator=dataset_gen,
                                      num_workers=8)

In [ ]:
splits = m3patients.dataset.random_splits([0.8, 0.9], random_seed=42, balanced='admissions')

In [ ]:
len(m3patients.subjects)

In [ ]:
m3patients.subjects[splits[0][0]]

In [ ]:
# m3patients.size_in_bytes() / 1024 ** 3

In [ ]:
# val_batch = m3patients.device_batch(splits[1])

In [ ]:
# tst_batch = m3patients.device_batch(splits[2])

In [ ]:
# val_batch.size_in_bytes() / 1024 ** 3, tst_batch.size_in_bytes() / 1024 ** 3

In [ ]:
# batch = m3patients.device_batch(splits[0][:32])

In [ ]:
# batch.size_in_bytes() / 1024 ** 3

In [ ]:
# len(batch.subjects)

In [ ]:
# batch.n_admissions()

In [ ]:
# batch.n_segments()

In [ ]:
# batch.n_obs_times()

### التدريب على نموذج المعادلات التفاضلية الاعتيادية العصبية


In [ ]:
from lib.ml import (ICENODE, ICENODEDimensions, 
                    GRU, GRUDimensions,
                    RETAIN, RETAINDimensions,
                    PatientEmbeddingDimensions, 
                    Trainer, TrainerReporting, OptimizerConfig, WarmupConfig,
                    TrainerConfig, ReportingConfig)
from lib.metric import  (CodeAUC, UntilFirstCodeAUC, AdmissionAUC,
                      CodeGroupTopAlarmAccuracy, LossMetric, ObsCodeLevelLossMetric)

import jax.random as jrandom

In [ ]:
emb_dims = PatientEmbeddingDimensions(dx=50, demo=0)
key = jrandom.PRNGKey(0)

def icenode_model():
    dims = ICENODEDimensions(mem=20, emb=emb_dims)
    return ICENODE(dims=dims, schemes=m3patients.schemes, 
                   demographic_vector_config=m3patients.config.demographic_vector,
                   key=key)

def gru_model():
    dims = GRUDimensions(emb=emb_dims)
    return GRU(dims=dims, schemes=m3patients.schemes, 
                   demographic_vector_config=m3patients.config.demographic_vector,
                   key=key)

def retain_model():
    dims = RETAINDimensions(mem_a=25, mem_b=25, emb=emb_dims)
    return RETAIN(dims=dims, schemes=m3patients.schemes, 
                   demographic_vector_config=m3patients.config.demographic_vector,
                   key=key)

models = {
    'rnk_dx_icenode': icenode_model(),
#     'rnk_dx_gru': gru_model(),
#     'rnk_dx_retain': retain_model()
}

In [ ]:
# res = m.batch_predict(m4inpatients.device_batch(), leave_pbar=True)

In [ ]:

trainer_conf = TrainerConfig(optimizer=OptimizerConfig(opt='adam', lr=1e-3),
                             epochs=150,
                             batch_size=32,
                             dx_loss='allpairs_sigmoid_rank')
trainer = Trainer(trainer_conf)

warmup = WarmupConfig(epochs=0.1, 
                      batch_size=8,
                      opt='adam', lr=1e-3, 
                      decay_rate=0.5)

loss_metric =  LossMetric(m3patients, 
                          dx_loss=('softmax_bce', 'balanced_focal_softmax_bce', 
                                   'balanced_focal_bce', 'allpairs_exp_rank', 'allpairs_hard_rank', 
                                   'allpairs_sigmoid_rank'))

metrics = [CodeAUC(m3patients), 
           AdmissionAUC(m3patients), 
           CodeGroupTopAlarmAccuracy(m3patients, n_partitions=5, 
                                     top_k_list=[3, 5, 10, 15, 20],
                                     train_split=splits[0]),
           loss_metric]




In [ ]:
res = {}
for name in models:
    model = models[name]
    print(name)
    reporting_conf = ReportingConfig(output_dir=name,
                                     console=True,
                                     model_stats=False,
                                     parameter_snapshots=True,
                                     config_json=True)
    reporting = TrainerReporting(reporting_conf, metrics=metrics)
    
    res[name] = trainer(model, m3patients, 
                  splits=splits,
                  reporting=reporting,
                  n_evals=100,
                  warmup_config=warmup,
                  continue_training=True)

###### 